<a href="https://colab.research.google.com/github/enzo-augusto/Regressao-Simbolica-Serial/blob/main/Regress%C3%A3o_Simb%C3%B3lica_Serial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dois arquivos de texto são gerados ao rodar esse código. Um com os pontos do gráfico gerado pelo polinômio encontrado (graph.txt) e outro com as informações relevantes sobre o resultado (resultado.txt).


In [ ]:
%%writefile Ep2.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <unistd.h>
#include <math.h>
#define gerMax 10000                     /**Interessante variar */
#define grauMax 10                       /**Interessante variar */
#define coefiMax 5                       /**Interessante variar */
#define pontosDominio 40                 /**Interessante variar */
#define tamanhoGeracao 100
#define inicio_dominio 0                 /**Interessante variar */
#define final_dominio 1                  /**Interessante variar */

#define ajusta_dominio (double)(final_dominio - inicio_dominio)/(double)pontosDominio

//Gera floats aleatorios dentro de um intervalo dado
double gera_random_float (double inicio_intervalo, double final_intervalo)
{
    return ((double) rand()/(double) RAND_MAX)* (final_intervalo - inicio_intervalo) + inicio_intervalo;
}

//Gera ints aleatorios dentro de um intervalo dado
int gera_random_int (int inicio_intervalo, int final_intervalo)
{
    return ( rand()%(final_intervalo - inicio_intervalo + 1) +  inicio_intervalo);
}

//Da o que precisamos para definir um polinomio. A variavel diferenca é a diferenca
//entre o polinomio e a funcao dada, e sera o que usaremos para ordenar os polinomios de uma geracao
 typedef struct{
    int grau;
    double coeficientes[grauMax + 1];
    double diferenca;
}polinomio;

//inicializa o polinomio com valores aleatorios e zera a diferenca
void inicializa_polinomio(polinomio* poli)
{
    (*poli).grau = gera_random_int(0, grauMax);

    for(int i = 0; i <= grauMax; i++)
    {
        (*poli).coeficientes[i] = 0;
    }

    for(int i = 0; i <= (*poli).grau; i++)
    {
        (*poli).coeficientes[i] = gera_random_float(-coefiMax, coefiMax);
    }

    (*poli).diferenca = 0;
}


//Devolve um ponteiro para um vetor estatico com a imagem do polinomio
static double* gera_grafico(polinomio* poli)
{
    static double imagem[pontosDominio];
    int grau = (*poli).grau;
    for(int i = 0; i < pontosDominio; i++)
    {
        imagem[i]  = 0;
    }

    for(int i = 1; i < pontosDominio; i++)
    {
        for(int j = 0; j <= grau; j++)
        {
            imagem[i] = imagem[i] + (*poli).coeficientes[j]*pow((i*ajusta_dominio),(double)j);
        }
    }

    return imagem;
}

//retorna a soma das distancias entre as imagens da funcao dada e do polinomio
double calcula_diferenca(double funcao_dada[], double imagem_poli[])
{
    double diferenca = 0;

    for(int i = 0; i < pontosDominio; i++)
    {
        diferenca = diferenca + fabs(funcao_dada[i] - imagem_poli[i]);
    }
    return diferenca;
}

//retorna um polinomio mutado.
polinomio mutacao(polinomio poli)
{
    //se a diferenca é pequena, os coeficientes dos polinomios sao alterados a partir de uma distribuicao normal.
    //O grau pode ser aumentado ou coeficientes podem zer zerados também
    if(poli.diferenca < 20)                                                    //Interessante variar
    {
        for(int i = 0; i <= poli.grau; i++)
        {
            double novo_coefic = 0;

            for(int j = 0; j < 10; j++)
            {
            novo_coefic = novo_coefic + gera_random_float(poli.coeficientes[i] - 1, poli.coeficientes[i] + 1);
            }

            poli.coeficientes[i] = novo_coefic/10;
        }

        if(poli.grau < grauMax && gera_random_int(1, 10) >= 8)
        {
            poli.coeficientes[poli.grau + 1] = 1;
            poli.grau++;
        }

        if(poli.grau > 0 && gera_random_int(1, 10) >= 5)
        {
        int coeficiente_zerado = gera_random_int(0, poli.grau);
        poli.coeficientes[coeficiente_zerado] = 0;
        }

    }else if( poli.diferenca >= 20 && poli.diferenca < 100){ //Se a diferenca e relativamente alta os coeficientes sao alterados de forma mais drastica
        for(int i = 0; i <= poli.grau; i++)
        {
            poli.coeficientes[i] = poli.coeficientes[i] + gera_random_float(-4, 4);
        }
    }else{  //Se a diferenca e muito grande o polinomio e descartado e outro aleatorio toma seu lugar
        inicializa_polinomio(&poli);
    }

    return poli;
}

//Ordena os polinomios dentro de um vetor a partir de suas diferencas
void ordena_geracao(polinomio poli[])
{
    polinomio poli_aux[tamanhoGeracao];
    poli_aux[0] = poli[0];
    for(int i = 1; i < tamanhoGeracao; i++){
        for(int contador = i; contador >= 1; contador--){
            if (poli_aux[contador - 1].diferenca > poli[i].diferenca){
                poli_aux[contador] = poli_aux[contador - 1];
                poli_aux[contador - 1] = poli[i];
            }
            else{
                poli_aux[contador] = poli[i];
                break;
            }
        }
    }
    for(int i = 0; i < tamanhoGeracao; i++)
    {
        poli[i] = poli_aux[i];
    }
}

//Recebe o vetor de polinomios ja ordenado. Cria uma nova geracao a partir de mutacoes dos nove melhores resultados
void nova_geracao(polinomio poli[])
{
    for(int i = 0; i < 9; i++ )
    {
        for(int j = 0; j < 10; j++ )
        {
            poli[(i + 1)*10 + j] = mutacao(poli[i]);
        }
    }

}

int main()
{
    srand(time(NULL)*getpid());
    int geracao = 0;

    float dif_min;

    float dif_esp = 0.1; /**Interessante variar */

    polinomio polinomios_vivos [tamanhoGeracao];

    FILE* resultado = fopen("resultado.txt", "w");
    if(resultado == NULL)
    {
        printf("Erro ao abrir arquivo de escrita.\n");
        return 1;
    }

    // inicializa a funcao que iremos aproximar
    double funcao[pontosDominio];
    for(int i = 1; i < pontosDominio; i++)
    {
        funcao[i] = (i*ajusta_dominio)*(i*ajusta_dominio);     /**Interessante variar. i deve sempre ser multiplicado pelo ajusta_dominio para agir como x */
    }

    //inicializa os polinomios do nosso vetor
    for(int i = 0; i < tamanhoGeracao; i++)
    {
        inicializa_polinomio(&polinomios_vivos[i]);
    }

    //gera os graficos dos polinomios e os compara com o da funcao que iremos aproximar
    for ( int i = 0; i < tamanhoGeracao; i++)
    {
        double* imagem_poli = gera_grafico(&polinomios_vivos[i]);
        polinomios_vivos[i].diferenca = calcula_diferenca(funcao, imagem_poli);
    }

    ordena_geracao(polinomios_vivos);
    dif_min = polinomios_vivos[0].diferenca;

    while(dif_min > dif_esp && geracao < gerMax)
    {
        nova_geracao(polinomios_vivos);

        for ( int i = 0; i < tamanhoGeracao; i++)
        {
            double* imagem_poli = gera_grafico(&polinomios_vivos[i]);
            polinomios_vivos[i].diferenca = calcula_diferenca(funcao, imagem_poli);
        }
        ordena_geracao(polinomios_vivos);
        dif_min = polinomios_vivos[0].diferenca;
        geracao++;
        printf( "%d \n", geracao);
    }

    fprintf(resultado, "Geracao: %d \n", geracao);
    fprintf(resultado, "Grau: %d \n", polinomios_vivos[0].grau);

    for(int i = 0; i <= polinomios_vivos[0].grau; i++)
    {
        fprintf(resultado, "Coeficiente %d, %lf \n", i, polinomios_vivos[0].coeficientes[i]);
    }

    fprintf(resultado, "Diferenca minima %lf \n", polinomios_vivos[0].diferenca);



    FILE* graph = fopen("graph.txt", "w");
    if(resultado == NULL)
    {
        printf("Erro ao abrir arquivo de escrita.\n");
        return 1;
    }

    double* imagem = gera_grafico(&polinomios_vivos[0]);

    for(int i = 0; i < pontosDominio; i++)
    {
        double dominio_certo = i*ajusta_dominio;
        fprintf(graph, "%lf    %lf \n", dominio_certo, *(imagem + i));
    }


    fclose(resultado);
    fclose(graph);
    return 0;
}

Writing Ep2.c


In [ ]:
!gcc Ep2.c -o Ep2.out -lm
!./Ep2.out

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 
